In [2]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import metrics, linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, r2_score
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv("housing.csv")
df.head()

,0.00632 18.00 2.310 0 0.5380 6.5750 65.20 4.0900 1 296.0 15.30 396.90 4.98 24.00
0,0.02731 0.00 7.070 0 0.4690 6.4210 78...
1,0.02729 0.00 7.070 0 0.4690 7.1850 61...
2,0.03237 0.00 2.180 0 0.4580 6.9980 45...
3,0.06905 0.00 2.180 0 0.4580 7.1470 54...
4,0.02985 0.00 2.180 0 0.4580 6.4300 58...


In [4]:
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns = boston.feature_names)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [8]:

scaler = preprocessing.MinMaxScaler()
x_scaled = scaler.fit_transform(df)
y = boston.target


In [20]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=0.35) 
lr=linear_model.LinearRegression()
lr.fit(X_train,y_train)
pred=lr.predict(X_test)
accuracy=lr.score(X_test,y_test)
print("Accuracy of test is ",accuracy*100)
io=np.sqrt(metrics.mean_squared_error(y_test,pred))
print("mean square error of test is ",io)


pred_val=lr.predict(X_val)
accuracy_val=lr.score(X_val,y_val)
print("Accuracy of val is ",accuracy*100)
io=np.sqrt(metrics.mean_squared_error(y_val,pred_val))
print("mean square error of val is ",io)

Accuracy of test is  61.86013596996644
mean square error of test is  6.002516027045947
Accuracy of val is  61.86013596996644
mean square error of val is  4.174849555268342


In [32]:
from sklearn.metrics import mean_squared_error, r2_score
list_mse_train=[]
list_mse_val=[]
list_deg=[]
for deg in range(1,8):
    poly_reg = PolynomialFeatures(degree=deg)
    X_poly = poly_reg.fit_transform(X_train)
    X_val_pol=poly_reg.transform(X_val)
    pol_reg = linear_model.LinearRegression()
    pol_reg.fit(X_poly, y_train)
    y_predict= pol_reg.predict(X_poly)
    list_deg.append(deg)
    list_mse_train.append(np.sqrt(mean_squared_error(y_train, y_predict)))
    list_mse_val.append(np.sqrt(mean_squared_error(y_val, pol_reg.predict(X_val_pol))))
print("The best degree in train is",list_deg[np.argmin(list_mse_train)]," with value ",list_mse_train[np.argmin(list_mse_train)]) 
print("The best degree in val is",list_deg[np.argmin(list_mse_val)]," with value ",list_mse_val[np.argmin(list_mse_val)]) 


The best degree in train is 4  with value  1.4258507992085474e-11
The best degree in val is 1  with value  4.174849555268324


In [34]:
err_val=[]
deg=[]
err_train=[]
alpha=[]
lambds=[0,0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
ridge = Ridge()
for i in lambds:
    alpha.append(i)
    poly = PolynomialFeatures(degree = 1)
    X_poly = poly.fit_transform(X_train)
    x_poly_val=poly.transform(X_val)
    pol=Ridge(alpha=i)
    pol.fit(X_poly, y_train) 
    y_pred=pol.predict(x_poly_val)
    mean_error=np.sqrt(mean_squared_error(y_val,y_pred))
    err_val.append(mean_error)
    y_train_pred=pol.predict(X_poly)
    mean_error_train=np.sqrt(mean_squared_error(y_train,y_train_pred))
    err_train.append(mean_error_train)
np.min(err_val), np.argmin(err_val)    

(4.174849555268326, 0)

In [37]:
from sklearn.linear_model import Lasso
err_val_lasso=[]
deg=[]
err_train_lasso=[]
alpha=[]
lambds=[0,0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
lasso = Lasso()
for i in lambds:
    alpha.append(i)
    poly = PolynomialFeatures(degree = 1)
    X_poly = poly.fit_transform(X_train)
    x_poly_val=poly.transform(X_val)
    pol=Lasso(alpha=i)
    pol.fit(X_poly, y_train) 
    y_pred=pol.predict(x_poly_val)
    mean_error=np.sqrt(mean_squared_error(y_val,y_pred))
    err_val_lasso.append(mean_error)
    y_train_pred=pol.predict(X_poly)
    mean_error_train=np.sqrt(mean_squared_error(y_train,y_train_pred))
    err_train_lasso.append(mean_error_train)
np.min(err_val_lasso), np.argmin(err_val_lasso)    

(4.174849555268319, 0)